In [1]:
"""
# Projeto de Classificação Binária com PCA e Árvores de Decisão
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.pipeline import Pipeline
import os

# # 1. Carregamento e Preparação dos Dados

file_path = 'sonar_dataset.csv'

if not os.path.exists(file_path):
    print(f"Erro: O arquivo '{file_path}' não foi encontrado.")
    print("Por favor, faça o upload do arquivo para o ambiente do notebook.")
else:
    df = pd.read_csv(file_path, header=None)

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    print("Dimensões dos dados de treino:", X_train.shape)
    print("Dimensões dos dados de teste:", X_test.shape)
    print("\nClasses e sua codificação:", list(label_encoder.classes_), "->", label_encoder.transform(label_encoder.classes_))
    print("\nDistribuição das classes no conjunto de treino:")
    print(pd.Series(y_train).value_counts(normalize=True))
    print("\nDistribuição das classes no conjunto de teste:")
    print(pd.Series(y_test).value_counts(normalize=True))

# # 2. Criação das Features com Análise de Componentes Principais (PCA)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    pca_analysis = PCA(random_state=42)
    pca_analysis.fit(X_train_scaled)
    explained_variance_ratio = pca_analysis.explained_variance_ratio_
    cumulative_explained_variance = np.cumsum(explained_variance_ratio)
    
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', linestyle='--')
    plt.title('Variância Explicada Cumulativa pelos Componentes Principais')
    plt.xlabel('Número de Componentes')
    plt.ylabel('Variância Explicada Cumulativa')
    plt.grid(True)
    plt.axhline(y=0.95, color='r', linestyle='-', label='95% de Variância Explicada')
    plt.legend(loc='best')
    plt.show()


    n_components_95 = np.where(cumulative_explained_variance >= 0.95)[0][0] + 1
    print(f"\nNúmero de componentes para reter 95% da variância: {n_components_95}")

    pca = PCA(n_components=n_components_95, random_state=42)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    print("\nDimensões dos dados de treino após PCA:", X_train_pca.shape)
    print("Dimensões dos dados de teste após PCA:", X_test_pca.shape)

# # 3. Modelo de ML: Árvore de Decisão e Avaliação com Validação Cruzada

    tree_clf_initial = DecisionTreeClassifier(random_state=42)

    scores = cross_val_score(tree_clf_initial, X_train_pca, y_train, cv=10, scoring='accuracy')

    print("\n# Avaliação Inicial com Validação Cruzada #")
    print(f"Acurácia Média da Validação Cruzada (10-fold): {scores.mean():.4f}")
    print(f"Desvio Padrão da Acurácia: {scores.std():.4f}")


# # 4. Busca Hiperparamétrica com GridSearchCV e Pruning

    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'ccp_alpha': [0.0, 0.005, 0.01, 0.015, 0.02] 
    }

    grid_search = GridSearchCV(
        DecisionTreeClassifier(random_state=42),
        param_grid,
        cv=10,
        scoring='accuracy',
        verbose=1,
        n_jobs=-1
    )

    grid_search.fit(X_train_pca, y_train)

    print("\n# Busca Hiperparamétrica com GridSearchCV #")
    print("Melhores Hiperparâmetros Encontrados:")
    print(grid_search.best_params_)
    print(f"\nMelhor Acurácia da Validação Cruzada: {grid_search.best_score_:.4f}")

    best_tree_clf = grid_search.best_estimator_


# # 5. Avaliação do Classificador Binário Final

    y_pred = best_tree_clf.predict(X_test_pca)
    y_pred_proba = best_tree_clf.predict_proba(X_test_pca)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    print("\n# Avaliação do Modelo Final no Conjunto de Teste #")
    print(f"Acurácia no Teste: {accuracy:.4f}")
    print(f"Precisão no Teste: {precision:.4f}")
    print(f"Recall (Sensibilidade) no Teste: {recall:.4f}")
    print(f"F1-Score no Teste: {f1:.4f}")
    print(f"Especificidade no Teste: {specificity:.4f}")

    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=['Rocha (0)', 'Mina (1)']))

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title('Matriz de Confusão')
    plt.ylabel('Classe Verdadeira')
    plt.xlabel('Classe Prevista')
    plt.show()

    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (área = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falsos Positivos')
    plt.ylabel('Taxa de Verdadeiros Positivos (Sensibilidade)')
    plt.title('Curva Característica de Operação do Receptor (ROC)')
    plt.legend(loc="lower right")
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'